In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [0]:
class ConvLN(tf.keras.Model):

  def __init__(self, filters, kernel_size):
    super(ConvLN, self).__init__()
    # Layer Normalization does same role with Bias
    self.conv = layers.Conv2D(filters, kernel_size, padding='same', use_bias=False)
    self.ln = layers.LayerNormalization()

  def __call__(self, x, training=False, mask=None):
    x = self.conv(x)
    # Layer Normalization does not need the training variable
    x = self.ln(x)
    return layers.Activation('relu')(x)

In [0]:
class Cnn(tf.keras.Model):
  
  def __init__(self):
    super(Cnn, self).__init__(self)
    self.conv1_1 = ConvLN(16, 3)
    self.conv1_2 = ConvLN(16, 3)
    self.pool1 = layers.MaxPool2D()

    self.conv2_1 = ConvLN(16, 3)
    self.conv2_2 = ConvLN(32, 3)
    self.pool2 = layers.MaxPool2D()

    self.conv3_1 = ConvLN(64, 3)
    self.conv3_2 = ConvLN(64, 3)

    self.flatten = layers.Flatten()
    self.dense1 = layers.Dense(1024, 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
    self.dense2 = layers.Dense(10, 'softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))
  def __call__(self, x, training=False, mask=None):
    x = self.conv1_1(x)
    x = self.conv1_2(x)
    x = self.pool1(x)

    x = self.conv2_1(x)
    x = self.conv2_2(x)
    x = self.pool2(x)

    x = self.conv3_1(x)
    x = self.conv3_2(x)

    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    return x

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
train_x = x_train/255.
test_x = x_test/255.
train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)
train_ds = tf.data.Dataset.from_tensor_slices((train_x, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, y_test)).batch(32).prefetch(1024)

170500096/170498071 [==============================] - 6s 0us/step


In [5]:
model = Cnn()
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=100)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 1563 steps, validate on 313 steps
Epoch 1/100
1563/1563 [==============================] - 24s 16ms/step - loss: 2.2256 - acc: 0.3845 - val_loss: 1.4110 - val_acc: 0.5333
Epoch 2/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.2958 - acc: 0.5800 - val_loss: 1.2038 - val_acc: 0.6183
Epoch 3/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.1269 - acc: 0.6437 - val_loss: 1.1409 - val_acc: 0.6437
Epoch 4/100
1563/1563 [==============================] - 17s 11ms/step - loss: 1.0330 - acc: 0.6790 - val_loss: 0.9883 - val_acc: 0.6965
Epoch 5/100
1563/1563 [==============================] - 18s 11ms/step - loss: 0.9666 - acc: 0.7027 - val_loss: 0.9876 - val_acc: 0.6891
Epoch 6/100
1563/1563 [==============================] - 18s 11ms/step - loss: 0.9138 - acc: 0.7226 - val_loss: 0.8980 - val_acc: 0.7252
Epoch 7/100
1563/1563 [==============================]